In [1]:
from data import *
from ml import *

#Model 
from xgboost import XGBClassifier

#AUC 
from sklearn.model_selection import cross_val_score


from sklearn.linear_model import LogisticRegression

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
df, X,y, X, X, y, y = dataset('baseline',28,1,'cv',0.25,'none')

In [3]:
#Include model score and logistic regression

In [4]:
def model(model,ttype,subset,death,treatment,split,size,missing): 
    #
    #-model: String, model to be trained: 'xgb','elasticnet','' 
    #-ttype: String, determines model to be trained 
    #-subset: String, determines subset: 'Lille','7day','Baseline','Paper'
    #-death: Integer, 30 days or 90 days
    #-treatment: Integer, 1 if treated and else 0
    #-split: String, determines split: 'train-test','mixed','cv'
    #-testsize: Float, Size of test set 
    #-missing: String, provides missingness mechanism: 'none','cc','ipw','mice'
    #
    # MODEL TRAINING
    #
    # Train XGB
    #
    # Read data
    df, X, y, X_train, X_test, y_train, y_test = dataset(subset,death,treatment,split,size,missing)
    #
    if model == 'xgb' :
        random_search = xgb_tuning_cv(X, y)
        training = XGBClassifier(**random_search.best_params_,objective='binary:logistic')
    #
    # Train EBM
    #
    elif model == 'gam':
        random_search = ebm_tuning_cv(X,y)
        training = ExplainableBoostingClassifier(**random_search.best_params_)
    #
    #Model Training
    #  
    if ttype == 'cv':
        #df, X,y = dataset(subset,death,treatment,split,size,missing)
        cv_aucs = cross_val_score(training,X,y,cv=10,scoring='roc_auc')
        #score = cross_val_score(training,X,y,cv=10,scoring='score')
        auc = np.mean(cv_aucs)
        
    else:
        #df, X, y, X_train, X_test, y_train, y_test = dataset(subset,death,treatment,split,size,missing)
        model_fit = model.fit(X_train, y_train)
        #score = model_fit.score(X_test, y_test)
        auc = roc_auc_score(y_test, model_fit.predict_proba(X_test)[:, 1])
        
    return auc
        
        
#Fix assignement of model 

In [ ]:
no_mis = []

for d in [28,90]:
    for x in ['xgb']: #,'gam'
        for y in ['baseline','7day','full']:
            print('death: '+ str(d))
            print()
            print('model: '+ x)
            print('Auc')
            print()
            no_mis.append(model(x,'cv',y,d,1,'cv',0.25,'none'))

In [9]:
no_mis

[0.8145876466528641,
 0.7530076777087646,
 0.8669220151828847,
 0.8033945824706693,
 0.7522985679779158,
 0.853798309178744,
 0.7331059218559218,
 0.7003228021978022,
 0.7562164224664224,
 0.7320619658119658,
 0.6961393467643466,
 0.7654410866910867]

In [19]:
#model('gam','cv',y,d,1,'cv',0.25,'ipw')

In [15]:
ipw_mis = []

for d in [28,90]:
    for x in ['xgb']:
        for y in ['baseline','7day','full']:
            print('death: '+ str(d))
            print()
            print('model: '+ x)
            print('Auc')
            print()
            ipw_mis.append(model(x,'cv',y,d,1,'cv',0.25,'ipw'))

death: 28

model: xgb
Auc



/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


death: 28

model: xgb
Auc



/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


death: 28

model: xgb
Auc



...

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


death: 90

model: xgb
Auc



/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


death: 90

model: xgb
Auc



/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [16]:
ipw_mis

[0.6670329670329671,
 0.7913866930171278,
 0.7340277777777777,
 0.7236645299145299,
 0.7875751879699248,
 0.7056547619047621]

In [18]:
cc_mis = []

for d in [28,90]:
    for x in ['gam','xgb']:
        for y in ['baseline','7day','full']:
            print('death: '+ str(d))
            print()
            print('model: '+ x)
            print('Auc')
            print()
            cc_mis.append(model(x,'cv',y,d,1,'cv',0.25,'cc'))

death: 28

model: gam
Auc



/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/interpret/glassbox/_ebm/_ebm.py", line 467, in fit
    raise ValueError(msg)
ValueError: greediness must be a percentage between 0.0 and 1.0 inclusive

  warnings.warn(some_fits_failed_message, FitFailedWarning)

death: 28

model: gam
Auc



/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/interpret/glassbox/_ebm/_ebm.py", line 467, in fit
    raise ValueError(msg)
ValueError: greediness must be a percentage between 0.0 and 1.0 inclusive

  warnings.warn(some_fits_failed_message, FitFailedWarning)

death: 28

model: gam
Auc



...

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


death: 90

model: xgb
Auc



/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


death: 90

model: xgb
Auc



/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


## Erstelle Programm 

# Tabellen erstellen

1. Erstelle Tabelle für XGB, GAM, ElasticNET, Scores



In [ ]:
mice_mis = []

for d in [28,90]:
    for x in ['gam','xgb']:
        for y in ['baseline','7day','full']:
            print('death: '+ str(d))
            print()
            print('model: '+ x)
            print('Auc')
            print()
            mice_mis.append(model(x,'cv',y,d,1,'cv',0.25,'mice'))

death: 28

model: gam
Auc



In [27]:
#Cross-Validation

np.random.seed(123)

clf = XGBClassifier(**random_search.best_params_,objective='binary:logistic')

cv_auc = cross_val_score(clf,X,y,cv=10,scoring='roc_auc')

np.mean(cv_auc)

0.9338929279576998

In [28]:
treated   = []
untreated = []

In [29]:
for treat in [0,1]:
    for sub in ['baseline','7day','full']:
        df, X, y, X_train, X_test, y_train, y_test = dataset(sub,28,treat,'train-test',0.25,'mice')
        cv_aucs = cross_val_score(clf,X,y,cv=10,scoring='roc_auc')
        cv_auc = np.mean(cv_aucs)
        if treat ==1:
            treated.append(cv_auc)
        else:
            untreated.append(cv_auc)
        
    

    

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [30]:
table28 = pd.DataFrame({'Treated':treated ,'Untreated':untreated})

In [31]:
table28

,Treated,Untreated
0,0.823783,0.820379
1,0.926551,0.924925
2,0.857438,0.874872
